In [1]:
from modelUXception import *
from data import *

vid="v010_5" #version id should match the file number 
# The final number can be [1:5] and denotes the cross-validation order
# Ran DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=45,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

batch_size=3
LR=1e-4
loss=dice_loss

train_gen = trainGenerator(batch_size,'afm_cell_512/train0'+vid[-1],'afmheight','labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'afm_cell_512/valid0'+vid[-1],'afmheight','labels',data_gen_args,save_to_dir = None)

# 26 images are used for training, 4 images for validating and 4 images for testing
train_steps = 24//batch_size
valid_steps = 6//batch_size

model = UXception()

opt = tf.keras.optimizers.Adam(LR)
metrics = ["acc", dice_coef]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files/unet_cell_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=20*train_steps, validation_steps=valid_steps, 
                    epochs=100, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Epoch 1/100
160/160 [==============================] - ETA: 0s - loss: 0.2496 - acc: 0.9204 - dice_coef: 0.7504Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.
160/160 [==============================] - 103s 506ms/step - loss: 0.2496 - acc: 0.9204 - dice_coef: 0.7504 - val_loss: 0.2018 - val_acc: 0.9294 - val_dice_coef: 0.7982

Epoch 00001: saving model to files\unet_cell_seg_v010_5.hdf5


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
160/160 [==============================] - 78s 485ms/step - loss: 0.1183 - acc: 0.9732 - dice_coef: 0.8817 - val_loss: 0.1062 - val_acc: 0.9621 - val_dice_coef: 0.8938

Epoch 00002: saving model to files\unet_cell_seg_v010_5.hdf5
Epoch 3/100
160/160 [==============================] - 79s 492ms/step - loss: 0.0936 - acc: 0.9782 - dice_coef: 0.9064 - val_loss: 0.0740 - val_acc: 0.9775 - val_dice_coef: 0.9260

Epoch 00003: saving model to files\unet_cell_seg_v010_5.hdf5
Epoch 4/100
160/160 [==============================] - 78s 489ms/step - loss: 0.0865 - acc: 0.9804 - dice_coef: 0.9135 - val_loss: 0.0739 - val_acc: 0.9838 - val_dice_coef: 0.9261

Epoch 00004: saving model to files\unet_cell_seg_v010_5.hdf5
Epoch 5/100
160/160 [==============================] - 78s 490ms/step - loss: 0.0813 - acc: 0.9816 - dice_coef: 0.9187 - val_loss: 0.0558 - val_acc: 0.9797 - val_dice_coef: 0.9442

Epoch 00005: saving model to files\unet_cell_seg_v010_5.hdf5
Epoch 6/100
160/160 [===========

### validate your model and save predicted results

In [3]:
validGene = testGenerator("afm_cell_512/valid0"+vid[-1]+"/predh")
model.load_weights("files/unet_cell_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,6,verbose=1)
saveResult("afm_cell_512/valid0"+vid[-1]+"/predh",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


6/6 [==============================] - 4s 172ms/step


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 255, 255, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 255, 255, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 255, 255, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________